In [96]:
# bigram_nn

import torch

import nltk
from nltk.tokenize import word_tokenize

xs = []
ys = []

with open('chatbot.txt') as f:
    sents = f.readlines()
    sent_words = [word_tokenize(words) for words in sents]
    for i, words in enumerate(sent_words):
        # words = [word for word in words if word.isalpha()]
        words = [word.lower() for word in words]
        words.insert(0, '<S>')
        words.append('<E>')
        sent_words[i] = words


print(sent_words[0])


word_set = set()
for sent in sent_words:
    for word in sent:
        word_set.add(word)

word_cnt = len(word_set)

word_cnt

['<S>', 'hi', ',', 'how', 'are', 'you', 'doing', '?', 'i', "'m", 'fine', '.', 'how', 'about', 'yourself', '?', '<E>']


2512

In [97]:
device = 'cuda:0'

# tensorize

word_to_idx = {word:i for i,word in enumerate(word_set)}
idx_to_word = {i:word for i,word in enumerate(word_set)}

# building bigram
bigram = torch.zeros((word_cnt, word_cnt), dtype=torch.float64)

for sent in sent_words:
    for i in range(len(sent)-1):
        first_idx = word_to_idx[sent[i]]
        second_idx = word_to_idx[sent[i+1]]
        # bigram[first_idx, second_idx] += 1

        # append to the input of nn
        xs.append(first_idx)
        ys.append(second_idx)

# torch.sum(bigram, dim=1)
# smoothing
# bigram += 1
# bigram = bigram / torch.sum(bigram, dim=1)

# APPLY SOFTMAX
# bigram = torch.softmax(bigram, dim=1)

# bigram[422]

# bigram[422] /= torch.sum(bigram[422])
# torch.sum(bigram[422])


In [98]:
import torch.nn.functional as F

xs = torch.tensor(xs)
ys = torch.tensor(ys)
xenc = F.one_hot(xs, num_classes=word_cnt).float().to(device)
yenc = F.one_hot(ys, num_classes=word_cnt).float().to(device)
xenc.shape

torch.Size([64466, 2512])

In [99]:
# # normal distribution
# # initialize weight of first layer

n_neuron = 27

# W = torch.randn((word_cnt, n_neuron))
# (xenc @ W).shape
# # the firing rate of each of the neurons to each of the inputs

In [100]:
# initialize weight of output layer
W_2 = torch.randn((n_neuron, word_cnt))

import torch.nn as nn

class Model(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.layer1 = nn.Linear(word_cnt, n_neuron)
        self.out_layer = nn.Linear(n_neuron, word_cnt)
    def forward(self, x):
        x = F.relu(self.layer1(x))
        return F.softmax(self.out_layer(x), dim=1)


model = Model().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.03)


In [101]:

for i in range(1000):
    optimizer.zero_grad()
    out = model(xenc)
    loss = F.cross_entropy(out, yenc)
    loss.backward()
    optimizer.step()
    if i % 10 == 0:
        print(loss)

tensor(7.8288, device='cuda:0', grad_fn=<DivBackward1>)
tensor(7.8246, device='cuda:0', grad_fn=<DivBackward1>)
tensor(7.7365, device='cuda:0', grad_fn=<DivBackward1>)
tensor(7.6774, device='cuda:0', grad_fn=<DivBackward1>)
tensor(7.6762, device='cuda:0', grad_fn=<DivBackward1>)
tensor(7.6762, device='cuda:0', grad_fn=<DivBackward1>)
tensor(7.6762, device='cuda:0', grad_fn=<DivBackward1>)
tensor(7.6760, device='cuda:0', grad_fn=<DivBackward1>)
tensor(7.6756, device='cuda:0', grad_fn=<DivBackward1>)
tensor(7.6752, device='cuda:0', grad_fn=<DivBackward1>)
tensor(7.6750, device='cuda:0', grad_fn=<DivBackward1>)
tensor(7.6732, device='cuda:0', grad_fn=<DivBackward1>)
tensor(7.6539, device='cuda:0', grad_fn=<DivBackward1>)
tensor(7.6535, device='cuda:0', grad_fn=<DivBackward1>)
tensor(7.6533, device='cuda:0', grad_fn=<DivBackward1>)
tensor(7.6532, device='cuda:0', grad_fn=<DivBackward1>)
tensor(7.6531, device='cuda:0', grad_fn=<DivBackward1>)
tensor(7.6530, device='cuda:0', grad_fn=<DivBack

In [102]:
# random sample and generate sent




# model(vec)

In [103]:
def generate_sentence(curr_word):
    while True:
        curr_idx = word_to_idx[curr_word]
        vec = torch.zeros((1, word_cnt), dtype=torch.float).to(device)
        vec[0,curr_idx] = 1
        prediction = idx_to_word[torch.argmax(model(vec)).item()]
        if prediction == '<E>':
            return
        else:
            print(prediction)
            curr_word = prediction

In [124]:
generate_sentence('pushed')

the
weather
.
